# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data1 = pd.read_csv(
    "Output/weather_data.csv")

# Visualize
weather_data.head1()

FileNotFoundError: [Errno 2] No such file or directory: 'Output/weather_data.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#configure gmaps
gmaps.configure(api_key=g_key)

max_humidity = weather_data["Humidity"].max()
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [ ]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
min_temp = weather_data.loc[weather_data['Max Temp']<80]
max_temp = min_temp.loc[min_temp['Max Temp']>70]
wind_speed = max_temp.loc[max_temp['Wind Speed']<10]
cloudiness_0 = wind_speed.loc[wind_speed['Cloudiness']==0]
weather_conditions = cloudiness_0.loc[cloudiness_0['Humidity']<70]
weather_conditions

# Convert to DataFrame
hotel_df = pd.DataFrame(weather_conditions)
hotel_df = weather_conditions_df.reset_index(drop=True)
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df['Hotel Name'] = ""

hotel_df

In [ ]:
hotel_name = []
city_name = []
country_name = []
lat = []
lng = []

count=1

print('Beginning Data Retrieval')     
print('-----------------------------')

    # set up a parameters dictionary
params = {
"keyword": "Hotel",
"radius": 5000,
"type": "Hotel",
"key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_name.append(name_address['results'][0]['name'])
        city_name.append(hotel_df["City"])
        country_name.append(hotel_df["Country"])
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
